## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-27-20-52-24 +0000,bbc,City trader sues UBS for $400m after rate-rigg...,https://www.bbc.com/news/articles/cz6ngvenxj6o...
1,2025-10-27-20-48-00 +0000,wsj,Amazon to Layoff Tens of Thousands of Corporat...,https://www.wsj.com/tech/amazon-to-layoff-tens...
2,2025-10-27-20-43-24 +0000,nyt,Live Updates: Hurricane Melissa Churns to Jama...,https://www.nytimes.com/live/2025/10/21/weathe...
3,2025-10-27-20-39-05 +0000,wapo,How Jack Smith’s strongest case against Donald...,https://www.washingtonpost.com/investigations/...
4,2025-10-27-20-35-27 +0000,nypost,Why many NYC Democrats will never vote for Zoh...,https://nypost.com/2025/10/27/us-news/why-many...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
29,trump,46
165,china,21
92,new,19
364,he,15
64,year,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
203,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...,133
243,2025-10-27-04-17-00 +0000,wsj,"Asian Markets Rise as U.S.-China Talks, Trump ...",https://www.wsj.com/finance/stocks/asian-marke...,105
251,2025-10-27-02-01-40 +0000,bbc,US and China agree framework of trade deal ahe...,https://www.bbc.com/news/articles/c1lqj5lz4geo...,99
244,2025-10-27-04-09-01 +0000,nypost,"US, China talks sketch out rare earths, tariff...",https://nypost.com/2025/10/27/world-news/us-ch...,96
125,2025-10-27-15-22-03 +0000,bbc,Trump does not rule out seeking third term - b...,https://www.bbc.com/news/articles/c797q57ple9o...,95


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
203,133,2025-10-27-09-37-17 +0000,nypost,Trump rules out running as Vance’s VP to gain ...,https://nypost.com/2025/10/27/us-news/trump-ru...
212,75,2025-10-27-09-01-23 +0000,wsj,Stock Market Today: Dow Hits New Record on U.S...,https://www.wsj.com/livecoverage/stock-market-...
98,61,2025-10-27-16-33-34 +0000,latimes,'CBS Evening News' co-anchor John Dickerson wi...,https://www.latimes.com/entertainment-arts/bus...
16,48,2025-10-27-20-17-00 +0000,wsj,"“Both political parties have made their point,...",https://www.wsj.com/politics/policy/union-pres...
118,46,2025-10-27-15-42-00 +0000,nypost,Treasury Secretary Bessent says he felt ‘pain’...,https://nypost.com/2025/10/27/business/treasur...
252,41,2025-10-27-01-32-03 +0000,nypost,Hecklers taunt Gov. Hochul with ‘Tax the Rich’...,https://nypost.com/2025/10/26/us-news/hecklers...
102,37,2025-10-27-16-28-00 +0000,wsj,A shortlist of candidates to be the next Fed c...,https://www.wsj.com/economy/central-banking/tr...
188,34,2025-10-27-11-01-00 +0000,startribune,America’s ‘inflation diet’ is starving junk fo...,https://www.startribune.com/junk-food-inflatio...
41,32,2025-10-27-19-04-27 +0000,bbc,Jamaica in path of 'life-threatening' category...,https://www.bbc.com/news/articles/c2dr0z57nygo...
233,30,2025-10-27-06-46-40 +0000,nypost,Trump congratulates Argentina’s Javier Milei a...,https://nypost.com/2025/10/27/world-news/argen...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
